In [ ]:
!pip install google-genai pydantic

In [2]:
import os
os.environ['GEMINI_API_KEY'] = 'MY_API_KEY'

In [3]:
import os
from google import genai
from pydantic import BaseModel, Field

# Initialize the Google GenAI client (uses GEMINI_API_KEY from environment)
client = genai.Client()
MODEL_ID = 'gemini-2.5-flash'

# --- 1. Define the Structured Schema for Root Decomposition ---

class DecompositionPlan(BaseModel):
    research_directive: str = Field(description="Instructions for the Research Subagent on what specific data to gather.")
    writing_directive: str = Field(description="Instructions for the Writing Subagent on how to format and synthesize the final report.")

# --- 2. Level 3: Lowest-Level Worker Agents (Execution) ---

def worker_a1_tech_researcher(topic: str) -> str:
    """Subagent A.1: Specializes in gathering technical specifications."""
    print("      -> [Worker A.1] Gathering technical data...")
    prompt = f"Provide 3 bullet points on the technical advancements of: {topic}"
    return client.models.generate_content(model=MODEL_ID, contents=prompt).text

def worker_a2_market_researcher(topic: str) -> str:
    """Subagent A.2: Specializes in gathering market and business impact data."""
    print("      -> [Worker A.2] Gathering market data...")
    prompt = f"Provide 3 bullet points on the market impact and business use cases of: {topic}"
    return client.models.generate_content(model=MODEL_ID, contents=prompt).text

def worker_b1_drafter(writing_directive: str, research_data: str) -> str:
    """Subagent B.1: Specializes in writing the initial rough draft."""
    print("      -> [Worker B.1] Drafting initial report...")
    prompt = f"""Draft a report based on this data.
    Directive: {writing_directive}
    Data: {research_data}"""
    return client.models.generate_content(model=MODEL_ID, contents=prompt).text

def worker_b2_editor(draft: str) -> str:
    """Subagent B.2: Specializes in formatting, editing, and final polish."""
    print("      -> [Worker B.2] Editing and polishing final report...")
    prompt = f"Edit the following draft for professional tone and perfect markdown formatting. Keep it concise:\n\n{draft}"
    return client.models.generate_content(model=MODEL_ID, contents=prompt).text

# --- 3. Level 2: Mid-Level Coordinator Agents (Delegation) ---

def task_a_research_coordinator(directive: str) -> str:
    """Task A Subagent: Decomposes the research directive and delegates to A.1 and A.2."""
    print("   -> [Task A Coordinator] Decomposing research tasks...")
    # In a fully dynamic system, this agent might use an LLM to decide WHICH workers to call.
    # For simplicity, we execute the predefined workers under its domain.
    tech_data = worker_a1_tech_researcher(directive)
    market_data = worker_a2_market_researcher(directive)

    print("   -> [Task A Coordinator] Synthesizing gathered research...")
    return f"--- Technical Research ---\n{tech_data}\n\n--- Market Research ---\n{market_data}"

def task_b_writing_coordinator(directive: str, research_data: str) -> str:
    """Task B Subagent: Manages the synthesis and drafting process via B.1 and B.2."""
    print("   -> [Task B Coordinator] Initiating drafting process...")

    # 1. Delegate to Drafter
    rough_draft = worker_b1_drafter(directive, research_data)

    # 2. Delegate to Editor
    final_report = worker_b2_editor(rough_draft)

    return final_report

# --- 4. Level 1: Root Coordinator Agent ---

def hierarchical_root_agent(user_prompt: str) -> str:
    """
    The top-level root agent.
    Receives a complex, ambiguous task and decomposes it into actionable directives
    for the mid-level coordinators.
    """
    print("-> [Root Coordinator] Receiving ambiguous task and planning decomposition...\n")

    # Use AI orchestration to formulate a decomposition plan
    planning_prompt = f"""You are the root manager for a complex research project.
    Decompose the user's high-level goal into two specific directives:
    1. A research directive detailing exactly what data needs to be gathered.
    2. A writing directive detailing how the final report should be structured.

    User Goal: {user_prompt}"""

    plan_response = client.models.generate_content(
        model=MODEL_ID,
        contents=planning_prompt,
        config={
            'response_mime_type': 'application/json',
            'response_schema': DecompositionPlan,
        },
    )
    plan = plan_response.parsed
    print(f"-> [Root Coordinator] Plan created.\n   Research Focus: {plan.research_directive[:50]}...\n   Writing Focus: {plan.writing_directive[:50]}...\n")

    # Delegate to Task A (Research)
    compiled_research = task_a_research_coordinator(plan.research_directive)
    print("\n-> [Root Coordinator] Task A (Research) Complete. Passing data to Task B (Writing)...\n")

    # Delegate to Task B (Writing)
    final_output = task_b_writing_coordinator(plan.writing_directive, compiled_research)

    print("\n-> [Root Coordinator] All tasks complete. Delivering final response.")
    return final_output

# --- Execution ---
if __name__ == "__main__":
    # Example ambiguous, open-ended problem
    open_ended_task = "Analyze the rise of solid-state batteries in electric vehicles and prepare an executive briefing."

    # Trigger the hierarchical root agent
    final_report = hierarchical_root_agent(open_ended_task)

    print("\n=== Final Executive Briefing ===")
    print(final_report)

-> [Root Coordinator] Receiving ambiguous task and planning decomposition...

-> [Root Coordinator] Plan created.
   Research Focus: Gather data on the current state of solid-state ba...
   Writing Focus: Structure the executive briefing with an introduct...

   -> [Task A Coordinator] Decomposing research tasks...
      -> [Worker A.1] Gathering technical data...
      -> [Worker A.2] Gathering market data...
   -> [Task A Coordinator] Synthesizing gathered research...

-> [Root Coordinator] Task A (Research) Complete. Passing data to Task B (Writing)...

   -> [Task B Coordinator] Initiating drafting process...
      -> [Worker B.1] Drafting initial report...
      -> [Worker B.2] Editing and polishing final report...

-> [Root Coordinator] All tasks complete. Delivering final response.

=== Final Executive Briefing ===
Here's the edited draft for professional tone and perfect markdown formatting, keeping it concise:

## Executive Briefing: The Solid-State Battery Revolution for Elec